#### Import Libraries 

In [33]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.nn import Linear, ReLU, LeakyReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout, MSELoss
from torch.optim import Adam, SGD

#### Split the data

In [34]:
from main_config import main_config

H5_TRAIN = main_config["train_dir"]
H5_VAL = main_config["val_dir"]
H5_TEST = main_config["test_dir"]


def create_hdf5_generator(file_path, batch_size):
    file = h5py.File(file_path)
    data_size = file['data'].shape[0]

    while True: # loop through the dataset indefinitely
        for i in np.arange(0, data_size, batch_size):
            data = file['data'][i:i+batch_size]
            labels = file['labels'][i:i+batch_size]
            # converting data into torch format
            data  = torch.from_numpy(data)
            # converting the lables into torch format
            labels = torch.from_numpy(labels)
            yield data, labels



#### Creating CNN model

In [35]:
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(21, 32, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(32),
            LeakyReLU(inplace=True),
            # Defining another 2D convolution layer
            Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(32),
            LeakyReLU(inplace=True)
        )

        self.linear_layers = Sequential(
            Linear(32 * 8 * 8, 37)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

#### Define optimizer and loss function 

In [ ]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

#### Training 

In [ ]:
import numpy as np
import time

batch_size = 32

train_hdf5_generator = create_hdf5_generator(H5_TRAIN, batch_size)
val_hdf5_generator = create_hdf5_generator(H5_VAL, batch_size)

epochs = 5
min_valid_loss = np.inf

train_losses = []
val_losses = []

for e in range(epochs):
    train_loss = 0.0
    model.train()
    with tqdm(train_hdf5_generator, unit='batch') as tepoch:
        for data, labels  in tepoch:
            if labels.shape[0] < batch_size: 
                break
            tepoch.set_description(f"Epoch {e}")
            # labels = labels.to(torch.float32)
            labels = labels.to(torch.long)

            # converting the data into GPU format
            if torch.cuda.is_available():
                data, labels = data.cuda(), labels.cuda()

            # clearing the Gradients of the model parameters
            optimizer.zero_grad()
            #prediction
            target = model(data)
            loss = criterion(target, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_losses.append(loss.item())

            tepoch.set_postfix(loss=loss.item())
            time.sleep(0.1)

    valid_loss = 0.0
    model.eval()
    with tqdm(val_hdf5_generator, unit='batch') as vepoch:
        for data, labels in vepoch:
            if labels.shape[0] < batch_size: 
                break
            # labels = labels.to(torch.float32)
            labels = labels.to(torch.long)
            # converting the data into GPU format
            if torch.cuda.is_available():
                    data, labels = data.cuda(), labels.cuda()
            target = model(data)
            loss = criterion(target, labels)
            valid_loss = loss.item() * data.size(0)
            val_losses.append(loss.item())
            tepoch.set_postfix(loss=loss.item())
            time.sleep(0.1)
    
    print(f'Epoch {e+1} \t\t Training Loss: {train_loss / batch_size} \t\t Validation Loss: {valid_loss / batch_size}')
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        # Saving State Dict
        torch.save(model.state_dict(), 'saved_model.pth')



In [ ]:
# plotting the training and validation loss
# plt.plot(train_losses, label='Training loss')
# plt.plot(val_losses, label='Validation loss')
# plt.legend()
# plt.show()